In [77]:
import torch
from torch import nn
import numpy as np
import os
import math
import cv2
import random

import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision import datasets
import numpy as np
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [78]:
noTumors = os.listdir(path='../dataset/no/')
yesTumors = os.listdir(path='../dataset/yes/')

In [79]:
print('no:', len(noTumors))
print('yes:', len(yesTumors))

print('Test size (no): ', math.ceil((0.2 * len(noTumors))))
print('Test size (yes): ', math.ceil((0.2 * len(yesTumors))))

no: 98
yes: 155
Test size (no):  20
Test size (yes):  31


In [80]:
def readImg(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (32, 32))
    return img

In [81]:
trainImages = []
trainLabels = []
testImages = []
testLabels = []

archive = []

testNo = math.ceil(0.2 * len(noTumors))
testYes = math.ceil(0.2 * len(yesTumors))


# for i in noTumors:
#     choice = random.choice(noTumors)
#     noTumors.remove(choice)
#     print(len(noTumors))
#     # images.append(readImg('../dataset/no/' + i))

while len(noTumors) > 0:

    if len(noTumors) < testNo:
        for i in noTumors:
            testLabels.append(0)
            testImages.append(readImg('../dataset/no/' + i))
        break

    choice = random.choice(noTumors)
    noTumors.remove(choice)
    trainImages.append(readImg('../dataset/no/' + choice))
    trainLabels.append(0)



while len(yesTumors) > 0:
    if len(yesTumors) < testYes:
        for i in yesTumors:
            testLabels.append(1)
            testImages.append(readImg('../dataset/yes/' + i))
        break
    
    choice = random.choice(yesTumors)
    yesTumors.remove(choice)
    trainImages.append(readImg('../dataset/yes/' + choice))
    trainLabels.append(1)

In [82]:
trainImages = np.asarray(trainImages)
trainLabels = np.asarray(trainLabels)
testImages = np.asarray(testImages)
testLabels = np.asarray(testLabels)

In [83]:
trainImages.shape

(204, 32, 32, 3)

In [84]:
class autoencoderT2(nn.Module):
    def __init__(self):
        super(autoencoderT2, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)  
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)


    def forward(self, x):
        
        x = torch.max_pool2d(torch.relu(self.conv1(x)), (2, 2))
        x = torch.max_pool2d(torch.relu(self.conv2(x)), (2, 2))
        x = torch.max_pool2d(torch.relu(self.conv3(x)), (2, 2))
                
        return x

In [85]:
def train(model, lossFunc, optim, data_loader, modelType=2):
  epochs = 10
  outputs = []

  for epoch in range(epochs):
    train_loss = 0.0 

    for image in data_loader:  
      
      # Reshaping the image to (-1, 784)
      if modelType == 1:
        image = image.reshape(-1, 28*28)
        
      # The gradients are set to zero,
      optim.zero_grad()
      # Output of Autoencoder
      reconstructed = model(image)
        
      # Calculating the loss function
      loss = lossFunc(reconstructed, image)
        
      # the the gradient is computed and stored.
      # .step() performs parameter update
      loss.backward()
      optim.step()

      train_loss += loss.item() * image.size(0)
        
    train_loss = train_loss/len(data_loader)
    print('Epoch: {} - Training loss: {:.4f}'.format(epoch, train_loss))
    outputs.append((epochs, image, reconstructed))
  return outputs

In [86]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

# Download the MNIST Dataset
# dataset = datasets.MNIST('./data/', train=True, download=True,
#                        transform=transform)

dataset = trainImages
  
# DataLoader is used to load the dataset 
# for training
loader = torch.utils.data.DataLoader(dataset, batch_size = 10, 
                        shuffle=True, num_workers=4)

loss_function = nn.MSELoss()

In [87]:
model1 = autoencoderT2()
sgd1 = torch.optim.SGD(model1.parameters(), lr = 0.01)
outputs1 = train(model1, loss_function, sgd1, loader)

# originalImg, recreatedImg = separateImages(outputs1)
# showResults(originalImg, recreatedImg)

RuntimeError: expected scalar type Byte but found Float